In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data =pd.read_csv('food_orders_new_delhi.csv')

In [ ]:
data.head()


In [ ]:
data.info()   # to show indexes, columns, data-types of each column, memory at once

In [ ]:
from datetime import datetime
# convert 'data and time' columns to datetime

data['Order Date and Time'] = pd.to_datetime(data['Order Date and Time'])
data['Delivery Date and Time'] = pd.to_datetime(data['Delivery Date and Time'])


In [ ]:
data.info()

In [ ]:
def extract_discount(discount_str):
    if isinstance(discount_str, str):
        if 'off' in discount_str:
            return float(discount_str.split(' ')[0])
        elif '%' in discount_str:
            return float(discount_str.split('%')[0])
        else:
            return 0.0
    else:
        return 0.0

data['Discount Percentage'] = data['Discounts and Offers'].apply(lambda x: extract_discount(x))
data['Discount Amount'] = data.apply(lambda x: (x['Order Value'] * x['Discount Percentage']/100) if '%' in str(x['Discounts and Offers']) else x['Discount Percentage'], axis=1)
data.head()

In [ ]:
data.info()

In [ ]:
# calculate total costs and revenue per order
# I think there are 3-4 comumns that will get the total cast and revenue
data['Total Costs'] = data['Delivery Fee'] + data['Payment Processing Fee'] + data['Discount Amount']
data['Revenue'] = data['Commission Fee']
data['Profit'] = data['Revenue'] - data['Total Costs']

# aggregate data to get overall metrics
total_orders = data.shape[0]
total_revenue = data['Revenue'].sum()
total_costs = data['Total Costs'].sum()
total_profit = data['Profit'].sum()

metrics = {
    'Total Orders': total_orders,
    'Total Revenue': total_revenue,
    'total Cost': total_costs,
    'total Profit': total_profit}

print(metrics)
metrics = pd.DataFrame({
    'Total Orders': total_orders,
    'Total Revenue': total_revenue,
    'total Cost': total_costs,
    'total Profit': total_profit}, index =["a"])
metrics


In [ ]:
data['Profit'].head(20)

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt

#histogram for profit per order
plt.hist(data['Profit'], bins=30, color='skyblue', edgecolor='black')
plt.title('Profit Distribution per Order in Food Delivery')
plt.xlabel('Profit')
plt.ylabel('Number of Orders')
plt.axvline(data['Profit'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
# pie chart for the proportion of total costs
costs_breakdown = data[['Delivery Fee',  'Payment Processing Fee', 'Discount Amount']].sum()
plt.figure(figsize=(7, 7))
plt.pie(costs_breakdown, labels=costs_breakdown.index, autopct='%1.f%%', startangle=140, colors=['tomato', 'gold', 'lightblue'])
plt.title('Proportion of Total Cost in food delivery')
plt.show()

In [ ]:
# bar chart for total revenue, costs, and profit
totals = ['Total Revenue', 'Total Costs', 'Total Profit']
values = [total_revenue, total_costs, total_profit]

plt.figure(figsize=(8, 6))
plt.bar(totals, values, color=['green', 'red', 'blue'])
plt.title('Total Revenue, Costs, and Profit')
plt.ylabel('Amount (INR)')
plt.show()

In [ ]:
total_revenue

In [ ]:
total_costs

In [ ]:
total_profit

In [ ]:
data['Commission Percentage'] = (data['Commission Fee'] / data['Order Value']) * 100
data['Commission Percentage'].mean()
data['Effective Discount Percentage'] = (data['Discount Amount'] / data['Order Value']) * 100
data['Effective Discount Percentage'].mean()

In [ ]:
# filter the dataset for profitable order
profitable_orders = data[data['Profit']>0]

# calculate the average commision percentage for profitable orders
profitable_orders['Commission Percentage'] = (profitable_orders['Commission Fee'] / profitable_orders['Order Value']) * 100

# calculte the average discount percentage for profitable orders
profitable_orders['Effective Discount Percentage'] = (profitable_orders['Discount Amount'] / profitable_orders['Order Value']) * 100

# calculate the new averages
new_avg_commission_percentage = profitable_orders['Commission Percentage'].mean()
new_avg_discount_percentage = profitable_orders['Effective Discount Percentage'].mean()

print(new_avg_commission_percentage, new_avg_discount_percentage)

In [ ]:
# filter the dataset for profitable orders
profitable_orders = data[data['Profit'] > 0]

# calculate the average commission percentage for profitable orders
profitable_orders['Commission Percentage'] = (profitable_orders['Commission Fee'] / profitable_orders['Order Value']) * 100

# calculate the average discount percentage for profitable orders
profitable_orders['Effective Discount Percentage'] = (profitable_orders['Discount Amount'] / profitable_orders['Order Value']) * 100

# calculate the new averages
new_avg_commission_percentage = profitable_orders['Commission Percentage'].mean()
new_avg_discount_percentage = profitable_orders['Effective Discount Percentage'].mean()

print(new_avg_commission_percentage, new_avg_discount_percentage)

In [ ]:
# simulate profitability with recommended discounts and commissions
recommended_commission_percentage = 30.0
recommended_discount_percentage = 6.0

# calculate the simulated commission fee and discount amount using recommended percentage
data['Simulated Commission Fee'] = data['Order Value'] * (recommended_commission_percentage / 100)
data['Simulated Discount Amount'] = data['Order Value'] * (recommended_discount_percentage / 100)

# recalculate total costs and profit with simulated values
data['Simulated Total Costs'] = (data['Delivery Fee'] +
                                 data['Payment Processing Fee'] +
                                 data['Simulated Discount Amount'])

data['Simulated Profit'] = (data['Simulated Commission Fee'] -
                            data['Simulated Total Costs'])

# visualizing the comparison
import seaborn as sns

plt.figure(figsize=(14, 7))

# actual profitability
sns.kdeplot(data['Profit'], label='Actual Profitability', fill=True, alpha=0.5, linewidth=2)

# simulated profitability
sns.kdeplot(data['Simulated Profit'], label='Estimated Profitablity with recommended Rates', fill=True, alpha=0.5, linewidth=2)

plt.title('Comparison of Profitability in Food Delivery: Actual vs. Recommended Discounts and Commissions')
plt.xlabel('Profit')
plt.ylabel('Density')
plt.legend(loc='upper left')
plt.show()



In [ ]:
data['Simulated Profit'].sum()

In [ ]:
data['Simulated Total Costs'].sum()

In [ ]:
data['Simulated Commission Fee'].sum()